In [1]:
# Clone Repository of InstantCity
!rm -rf ./InstantCity/
!git clone https://github.com/ualsg/InstantCity

Klone nach 'InstantCity'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 166 (delta 6), reused 3 (delta 3), pack-reused 148
Empfange Objekte: 100% (166/166), 11.39 MiB | 24.05 MiB/s, fertig.
Löse Unterschiede auf: 100% (24/24), fertig.


In [4]:
# Change working directory to InstantCity
%cd InstantCity

/hdd/repository/InstantCITY/InstantCity


In [3]:
# What's inside
!ls

data		 images       options	   run_engine.py  util
environment.yml  LICENSE.txt  __pycache__  test.py
fid.py		 models       README.md    train.py


In [4]:
!cat ./README.md


<p align="center">
  <a href="https://ual.sg/">
    <img src="images/logo.jpg" alt="Logo">
  </a>
  <h3 align="center">InstantCITY: Synthesising morphologically accurate geospatial data for urban form analysis, transfer, and quality control</h3>
  <a >
    <img src="images/Pipeline.png" alt="Logo">
  </a>
</p>

This is the official repo of InstantCITY, a Geospatial Data Translation model for urban form analysis, transfer, and quality control.

## Running InstantCITY 

### 1. Install prerequisites

Use `environment.yml` to create a conda environment for GANmapper

  ```sh
  conda env create -f environment.yml
  conda activate IC
  ```

### 2. Download weights
The weights files are available on figshare in the Checkpoints folder.

```https://doi.org/10.6084/m9.figshare.15103128.v1```

Place the `Checkpoints` folder in the repo.
### 3. Prediction
Predictions can be carried out by running the following sample code. The name of the city depends on the name of each dataset.

 ```sh
 python 

In [5]:
!python --version

Python 3.11.3


In [6]:
# !pip install condacolab
# import condacolab
# condacolab.install()
# !conda --version
# !which conda

In [7]:
# install modules
!pip install pathlib
!pip install dominate
!pip install scipy
!pip install torch
!pip install Pillow
!pip install torchvision

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
error: externally-managed-environment

× This environment is externally managed
╰

In [8]:
# Connect to Google Drive to retreive GANmapper_Data
# from google.colab import drive
# drive.mount('/content/drive')
# !ls /content/drive/MyDrive/GANmapper_Data/

In [9]:
# Copy the testing data from GANmapper_Data
# !cp -r /content/drive/MyDrive/GANmapper_Data/checkpoints /content/InstantCity/
# !cp -r /content/drive/MyDrive/GANmapper_Data/datasets /content/InstantCity/

In [10]:
# Unzip GANmapper data
!unzip -q -o ../GANmapper\ Data.zip
# And move it to the parent folder
!cp -r ./GANmapper\ Data/checkpoints .
!cp -r ./GANmapper\ Data/datasets .

In [11]:
# What's inside datasets?
!ls ./datasets
!ls ./datasets/Exp4/

'Data Processing.ipynb'   Exp4
Jakarta  Jakarta17  LA	Paris  Singapore


In [12]:
# What's inside checkpoints?
!ls ./checkpoints/
!ls ./checkpoints/Exp3/
!ls ./checkpoints/Exp3/Singapore/

Exp1  Exp2  Exp3
Jakarta  LA  Singapore
latest_net_D.pth  loss_log.txt	train_opt.txt
latest_net_G.pth  test_opt.txt	web


In [10]:
# Copy modified python-scripts, to fix mistakes in the repo
!cp ../modified_files/train.py ./
!cp ../modified_files/test.py ./
!cp ../modified_files/pix2pixHD_model.py ./models/
!cp ../modified_files/models.py ./models/
!cp ../modified_files/image_folder.py ./data/
!cp ../modified_files/aligned_dataset.py ./data/

In [11]:
# What's in the test script?
!cat train.py

import time
import os
import numpy as np
import torch
from torch.autograd import Variable
from collections import OrderedDict
from subprocess import call
import fractions
from torch.cuda.amp import autocast, GradScaler


def gcd(a, b):
    """Calculate the Greatest Common Divisor of a and b.

    Unless b==0, the result will have the same sign as b (so that when
    b is divided by it, the result comes out positive).
    """
    while b:
        a, b = b, a%b
    return a

def lcm(a,b): return abs(a * b)/gcd(a,b) if a and b else 0

from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

if __name__=="__main__":
    scaler = GradScaler()
    torch.cuda.empty_cache()
    opt = TrainOptions().parse()
    iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')
    if opt.continue_train:
        try:
            start_epoch, epoch_iter = np

In [12]:
# What are the test's parameters?
!cat ./options/train_options.py

from .base_options import BaseOptions

class TrainOptions(BaseOptions):
    def initialize(self):
        BaseOptions.initialize(self)
        # for displays
        self.parser.add_argument('--display_freq', type=int, default=100, help='frequency of showing training results on screen')
        self.parser.add_argument('--print_freq', type=int, default=100, help='frequency of showing training results on console')
        self.parser.add_argument('--save_latest_freq', type=int, default=1000, help='frequency of saving the latest results')
        self.parser.add_argument('--save_epoch_freq', type=int, default=10, help='frequency of saving checkpoints at the end of epochs')        
        self.parser.add_argument('--no_html', action='store_true', help='do not save intermediate training results to [opt.checkpoints_dir]/[opt.name]/web/')
        self.parser.add_argument('--debug', action='store_true', help='only do one epoch and displays at each iteration')

        # for training
        

In [16]:
# Create the training folder structure
!rm -r ./training_data
!mkdir ./training_data
!mkdir ./training_data/train_A
!mkdir ./training_data/train_B

In [21]:
# Copy training images of Singapore to the training directory
!ls ./datasets/Exp4/
!cp -r ./datasets/Exp4/Singapore/Source/* ./training_data/train_A/
!cp -r ./datasets/Exp4/Singapore/Target/* ./training_data/train_B/

Jakarta  Jakarta17  LA	Paris  Singapore


In [17]:
# Copy training images of Allermöhe to the training directory
!cp -r ../data_training/allermoehe_source/* ./training_data/train_A/
!cp -r ../data_training/allermoehe_target/* ./training_data/train_B/

In [20]:
# Run the training of a model
!python train.py --name custom_model --dataroot ./training_data --debug
# Remember to copy the model after training to a permanent location, that won't get deleted (oudside of InstantCity/InstantCity/)

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: ./training_data
debug: True
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: custom_model
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 60
niter_decay: 40
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: 1
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use_

(epoch: 564, iters: 25, time: 0.296) G_GAN: 0.644 G_GAN_Feat: 5.035 G_VGG: 5.147 D_real: 0.081 D_fake: 0.439 
End of epoch 564 / 5820 	 Time Taken: 7 sec
End of epoch 565 / 5820 	 Time Taken: 7 sec
End of epoch 566 / 5820 	 Time Taken: 7 sec
End of epoch 567 / 5820 	 Time Taken: 7 sec
(epoch: 568, iters: 25, time: 0.295) G_GAN: 2.346 G_GAN_Feat: 3.865 G_VGG: 4.662 D_real: 0.582 D_fake: 0.053 
End of epoch 568 / 5820 	 Time Taken: 7 sec
End of epoch 569 / 5820 	 Time Taken: 7 sec
End of epoch 570 / 5820 	 Time Taken: 7 sec
End of epoch 571 / 5820 	 Time Taken: 7 sec
(epoch: 572, iters: 25, time: 0.293) G_GAN: 1.728 G_GAN_Feat: 4.204 G_VGG: 2.396 D_real: 0.163 D_fake: 0.156 
End of epoch 572 / 5820 	 Time Taken: 7 sec
End of epoch 573 / 5820 	 Time Taken: 7 sec
End of epoch 574 / 5820 	 Time Taken: 7 sec
End of epoch 575 / 5820 	 Time Taken: 7 sec
(epoch: 576, iters: 25, time: 0.293) G_GAN: 1.577 G_GAN_Feat: 2.328 G_VGG: 2.151 D_real: 0.632 D_fake: 0.063 
End of epoch 576 / 5820 	 Time T

End of epoch 676 / 5820 	 Time Taken: 7 sec
End of epoch 677 / 5820 	 Time Taken: 7 sec
End of epoch 678 / 5820 	 Time Taken: 7 sec
End of epoch 679 / 5820 	 Time Taken: 7 sec
(epoch: 680, iters: 25, time: 0.307) G_GAN: 1.880 G_GAN_Feat: 2.279 G_VGG: 2.819 D_real: 0.959 D_fake: 0.184 
saving the latest model (epoch 680, total_steps 17000)
End of epoch 680 / 5820 	 Time Taken: 15 sec
End of epoch 681 / 5820 	 Time Taken: 7 sec
End of epoch 682 / 5820 	 Time Taken: 7 sec
End of epoch 683 / 5820 	 Time Taken: 7 sec
(epoch: 684, iters: 25, time: 0.296) G_GAN: 0.648 G_GAN_Feat: 4.851 G_VGG: 4.150 D_real: 0.213 D_fake: 0.463 
End of epoch 684 / 5820 	 Time Taken: 7 sec
End of epoch 685 / 5820 	 Time Taken: 7 sec
End of epoch 686 / 5820 	 Time Taken: 7 sec
End of epoch 687 / 5820 	 Time Taken: 7 sec
(epoch: 688, iters: 25, time: 0.292) G_GAN: 0.467 G_GAN_Feat: 1.387 G_VGG: 1.579 D_real: 0.402 D_fake: 0.586 
End of epoch 688 / 5820 	 Time Taken: 7 sec
End of epoch 689 / 5820 	 Time Taken: 7 se

End of epoch 788 / 5820 	 Time Taken: 7 sec
End of epoch 789 / 5820 	 Time Taken: 7 sec
End of epoch 790 / 5820 	 Time Taken: 7 sec
End of epoch 791 / 5820 	 Time Taken: 7 sec
(epoch: 792, iters: 25, time: 0.295) G_GAN: 1.472 G_GAN_Feat: 4.824 G_VGG: 3.848 D_real: 0.074 D_fake: 0.438 
End of epoch 792 / 5820 	 Time Taken: 7 sec
End of epoch 793 / 5820 	 Time Taken: 7 sec
End of epoch 794 / 5820 	 Time Taken: 7 sec
End of epoch 795 / 5820 	 Time Taken: 7 sec
(epoch: 796, iters: 25, time: 0.296) G_GAN: 0.918 G_GAN_Feat: 3.242 G_VGG: 3.407 D_real: 0.237 D_fake: 0.611 
End of epoch 796 / 5820 	 Time Taken: 7 sec
End of epoch 797 / 5820 	 Time Taken: 7 sec
End of epoch 798 / 5820 	 Time Taken: 7 sec
End of epoch 799 / 5820 	 Time Taken: 7 sec
(epoch: 800, iters: 25, time: 0.297) G_GAN: 1.505 G_GAN_Feat: 2.110 G_VGG: 1.543 D_real: 0.198 D_fake: 0.068 
saving the latest model (epoch 800, total_steps 20000)
End of epoch 800 / 5820 	 Time Taken: 16 sec
saving the model at the end of epoch 800, 

End of epoch 900 / 5820 	 Time Taken: 7 sec
saving the model at the end of epoch 900, iters 22500
End of epoch 901 / 5820 	 Time Taken: 7 sec
End of epoch 902 / 5820 	 Time Taken: 7 sec
End of epoch 903 / 5820 	 Time Taken: 7 sec
(epoch: 904, iters: 25, time: 0.294) G_GAN: 1.629 G_GAN_Feat: 4.268 G_VGG: 3.946 D_real: 0.299 D_fake: 0.058 
End of epoch 904 / 5820 	 Time Taken: 7 sec
End of epoch 905 / 5820 	 Time Taken: 7 sec
End of epoch 906 / 5820 	 Time Taken: 7 sec
End of epoch 907 / 5820 	 Time Taken: 7 sec
(epoch: 908, iters: 25, time: 0.295) G_GAN: 1.934 G_GAN_Feat: 3.672 G_VGG: 3.470 D_real: 0.137 D_fake: 0.025 
End of epoch 908 / 5820 	 Time Taken: 7 sec
End of epoch 909 / 5820 	 Time Taken: 7 sec
End of epoch 910 / 5820 	 Time Taken: 7 sec
End of epoch 911 / 5820 	 Time Taken: 7 sec
(epoch: 912, iters: 25, time: 0.296) G_GAN: 2.324 G_GAN_Feat: 3.737 G_VGG: 4.468 D_real: 1.570 D_fake: 0.738 
End of epoch 912 / 5820 	 Time Taken: 7 sec
End of epoch 913 / 5820 	 Time Taken: 7 sec


...

End of epoch 5303 / 5820 	 Time Taken: 7 sec
(epoch: 5304, iters: 25, time: 0.281) G_GAN: 1.843 G_GAN_Feat: 2.169 G_VGG: 2.099 D_real: 0.016 D_fake: 0.011 
End of epoch 5304 / 5820 	 Time Taken: 8 sec
End of epoch 5305 / 5820 	 Time Taken: 7 sec
End of epoch 5306 / 5820 	 Time Taken: 7 sec
End of epoch 5307 / 5820 	 Time Taken: 7 sec
(epoch: 5308, iters: 25, time: 0.282) G_GAN: 2.206 G_GAN_Feat: 3.116 G_VGG: 3.253 D_real: 0.009 D_fake: 0.017 
End of epoch 5308 / 5820 	 Time Taken: 8 sec
End of epoch 5309 / 5820 	 Time Taken: 7 sec
End of epoch 5310 / 5820 	 Time Taken: 7 sec
End of epoch 5311 / 5820 	 Time Taken: 7 sec
(epoch: 5312, iters: 25, time: 0.285) G_GAN: 2.054 G_GAN_Feat: 2.689 G_VGG: 2.795 D_real: 0.007 D_fake: 0.006 
End of epoch 5312 / 5820 	 Time Taken: 8 sec
End of epoch 5313 / 5820 	 Time Taken: 7 sec
End of epoch 5314 / 5820 	 Time Taken: 7 sec
End of epoch 5315 / 5820 	 Time Taken: 7 sec
(epoch: 5316, iters: 25, time: 0.284) G_GAN: 1.652 G_GAN_Feat: 2.271 G_VGG: 2.119 

End of epoch 5412 / 5820 	 Time Taken: 8 sec
End of epoch 5413 / 5820 	 Time Taken: 6 sec
End of epoch 5414 / 5820 	 Time Taken: 7 sec
End of epoch 5415 / 5820 	 Time Taken: 7 sec
(epoch: 5416, iters: 25, time: 0.283) G_GAN: 1.995 G_GAN_Feat: 2.845 G_VGG: 2.613 D_real: 0.006 D_fake: 0.008 
End of epoch 5416 / 5820 	 Time Taken: 8 sec
End of epoch 5417 / 5820 	 Time Taken: 7 sec
End of epoch 5418 / 5820 	 Time Taken: 7 sec
End of epoch 5419 / 5820 	 Time Taken: 7 sec
(epoch: 5420, iters: 25, time: 0.284) G_GAN: 1.866 G_GAN_Feat: 4.049 G_VGG: 2.590 D_real: 0.004 D_fake: 0.010 
End of epoch 5420 / 5820 	 Time Taken: 8 sec
End of epoch 5421 / 5820 	 Time Taken: 7 sec
End of epoch 5422 / 5820 	 Time Taken: 7 sec
End of epoch 5423 / 5820 	 Time Taken: 7 sec
(epoch: 5424, iters: 25, time: 0.284) G_GAN: 1.946 G_GAN_Feat: 3.651 G_VGG: 3.728 D_real: 0.006 D_fake: 0.005 
End of epoch 5424 / 5820 	 Time Taken: 8 sec
End of epoch 5425 / 5820 	 Time Taken: 7 sec
End of epoch 5426 / 5820 	 Time Taken

End of epoch 5523 / 5820 	 Time Taken: 7 sec
(epoch: 5524, iters: 25, time: 0.282) G_GAN: 2.049 G_GAN_Feat: 2.154 G_VGG: 2.146 D_real: 0.019 D_fake: 0.007 
End of epoch 5524 / 5820 	 Time Taken: 8 sec
End of epoch 5525 / 5820 	 Time Taken: 7 sec
End of epoch 5526 / 5820 	 Time Taken: 7 sec
End of epoch 5527 / 5820 	 Time Taken: 7 sec
(epoch: 5528, iters: 25, time: 0.283) G_GAN: 2.011 G_GAN_Feat: 2.769 G_VGG: 2.727 D_real: 0.005 D_fake: 0.005 
End of epoch 5528 / 5820 	 Time Taken: 8 sec
End of epoch 5529 / 5820 	 Time Taken: 7 sec
End of epoch 5530 / 5820 	 Time Taken: 7 sec
End of epoch 5531 / 5820 	 Time Taken: 7 sec
(epoch: 5532, iters: 25, time: 0.284) G_GAN: 2.173 G_GAN_Feat: 3.344 G_VGG: 2.918 D_real: 0.005 D_fake: 0.011 
End of epoch 5532 / 5820 	 Time Taken: 8 sec
End of epoch 5533 / 5820 	 Time Taken: 7 sec
End of epoch 5534 / 5820 	 Time Taken: 7 sec
End of epoch 5535 / 5820 	 Time Taken: 7 sec
(epoch: 5536, iters: 25, time: 0.283) G_GAN: 1.997 G_GAN_Feat: 2.689 G_VGG: 2.650 

End of epoch 5633 / 5820 	 Time Taken: 7 sec
End of epoch 5634 / 5820 	 Time Taken: 7 sec
End of epoch 5635 / 5820 	 Time Taken: 7 sec
(epoch: 5636, iters: 25, time: 0.283) G_GAN: 1.773 G_GAN_Feat: 2.746 G_VGG: 2.626 D_real: 0.007 D_fake: 0.025 
End of epoch 5636 / 5820 	 Time Taken: 8 sec
End of epoch 5637 / 5820 	 Time Taken: 7 sec
End of epoch 5638 / 5820 	 Time Taken: 7 sec
End of epoch 5639 / 5820 	 Time Taken: 7 sec
(epoch: 5640, iters: 25, time: 0.284) G_GAN: 2.124 G_GAN_Feat: 2.642 G_VGG: 2.776 D_real: 0.026 D_fake: 0.008 
saving the latest model (epoch 5640, total_steps 141000)
End of epoch 5640 / 5820 	 Time Taken: 17 sec
End of epoch 5641 / 5820 	 Time Taken: 7 sec
End of epoch 5642 / 5820 	 Time Taken: 7 sec
End of epoch 5643 / 5820 	 Time Taken: 7 sec
(epoch: 5644, iters: 25, time: 0.283) G_GAN: 1.491 G_GAN_Feat: 2.711 G_VGG: 2.956 D_real: 0.011 D_fake: 0.093 
End of epoch 5644 / 5820 	 Time Taken: 8 sec
End of epoch 5645 / 5820 	 Time Taken: 7 sec
End of epoch 5646 / 5820

(epoch: 5744, iters: 25, time: 0.283) G_GAN: 1.865 G_GAN_Feat: 4.057 G_VGG: 2.582 D_real: 0.007 D_fake: 0.015 
End of epoch 5744 / 5820 	 Time Taken: 8 sec
End of epoch 5745 / 5820 	 Time Taken: 7 sec
End of epoch 5746 / 5820 	 Time Taken: 7 sec
End of epoch 5747 / 5820 	 Time Taken: 7 sec
(epoch: 5748, iters: 25, time: 0.283) G_GAN: 1.873 G_GAN_Feat: 2.228 G_VGG: 2.342 D_real: 0.029 D_fake: 0.011 
End of epoch 5748 / 5820 	 Time Taken: 8 sec
End of epoch 5749 / 5820 	 Time Taken: 7 sec
End of epoch 5750 / 5820 	 Time Taken: 7 sec
End of epoch 5751 / 5820 	 Time Taken: 7 sec
(epoch: 5752, iters: 25, time: 0.282) G_GAN: 2.286 G_GAN_Feat: 3.792 G_VGG: 3.325 D_real: 0.007 D_fake: 0.025 
End of epoch 5752 / 5820 	 Time Taken: 8 sec
End of epoch 5753 / 5820 	 Time Taken: 7 sec
End of epoch 5754 / 5820 	 Time Taken: 7 sec
End of epoch 5755 / 5820 	 Time Taken: 7 sec
(epoch: 5756, iters: 25, time: 0.283) G_GAN: 1.614 G_GAN_Feat: 2.432 G_VGG: 2.961 D_real: 0.104 D_fake: 0.058 
End of epoch 575

In [14]:
# Copy Test-model
!mkdir ./checkpoints/SG15
print("Copying data...")
!cp -r ./checkpoints/Exp3/Singapore/* ./checkpoints/SG15/
print("Copy complete.")
!ls ./checkpoints/SG15/

Copying data...
Copy complete.
latest_net_D.pth  loss_log.txt	train_opt.txt
latest_net_G.pth  test_opt.txt	web


In [23]:
# Copy Test-dataset
!mkdir ./test
!rm -r ./test/Test
!mkdir ./test/Test
!ls ./datasets/Exp4/
print("Copying data...")
!cp -r ./datasets/Exp4/Paris/Source/* ./test/Test
print("Copy complete.")
!ls ./test/Test/

mkdir: das Verzeichnis „./test“ kann nicht angelegt werden: Die Datei existiert bereits
Jakarta  Jakarta17  LA	Paris  Singapore
Copying data...
Copy complete.
16


In [24]:
# Copy Oberbillwerder-dataset
!mkdir ./test
!rm -r ./test/oberbillwerder
!mkdir ./test/oberbillwerder
!ls ../data_training/oberbillwerder_source
print("Copying data...")
!cp -r ../data_training/oberbillwerder_source/* ./test/oberbillwerder
print("Copy complete.")
!ls ./test/oberbillwerder/

mkdir: das Verzeichnis „./test“ kann nicht angelegt werden: Die Datei existiert bereits
rm: das Entfernen von './test/oberbillwerder' ist nicht möglich: Datei oder Verzeichnis nicht gefunden
15  16
Copying data...
Copy complete.
15  16


In [21]:
# Available models
!ls ./checkpoints/

custom_model  Exp1  Exp2  Exp3	SG15


In [25]:
# Available test-datasets
!ls ./test/

oberbillwerder	Test


In [27]:
# Testing of the model
# --------------------

# Remember:
# The model must be available at "./checkpoints/{name}/"
# The the test images must be available at "./test/{dataroot}"
# !python test.py --name {name} --dataroot ./test/{dataroot}

!rm -r ./fake\\16/
!python test.py --name custom_model --dataroot ./test/oberbillwerder
!ls ./fake\\16/

rm: das Entfernen von './fake\16/' ist nicht möglich: Datei oder Verzeichnis nicht gefunden
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./test/oberbillwerder
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 5000
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: custom_model
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: 1
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_

In [38]:
!ls ./fake\\16/
!ls ./fake\\16/33198/
# from PIL import Image
# im = Image.open("./fake\\16/33198/22542.png")
# im.show()

33198  33199  33200  33201  33202  33203  33204  33205
22542.png  22544.png  22546.png  22548.png  22550.png
22543.png  22545.png  22547.png  22549.png
